# Judge Bias Detection Tutorial

## Cost Warning

- **DEMO MODE**: $0.50-1.00 (30 comparisons for bias detection)
- **FULL MODE**: $2.00-3.00 (100 comparisons for statistical significance)

## Learning Objectives

By the end of this tutorial, you will:

1. Detect self-preference bias (judges favoring their own model's outputs)
2. Detect position bias (judges favoring first or second position in A/B tests)
3. Detect verbosity bias (judges favoring longer responses)
4. Visualize bias patterns with statistical tests
5. Implement mitigation strategies (position swapping, length normalization)
6. Understand when biases invalidate evaluation results

In [ ]:
# Configuration
DEMO_MODE = True  # Set to False for full dataset
NUM_COMPARISONS = 10 if DEMO_MODE else 40

print(f"Running in {'DEMO' if DEMO_MODE else 'FULL'} mode")
print(f"Testing {NUM_COMPARISONS} comparisons per bias type = {NUM_COMPARISONS * 3} total evaluations")

In [ ]:
# Imports
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import List, Dict, Any, Tuple
from scipy import stats
from dotenv import load_dotenv
import litellm

# Add backend to path
sys.path.insert(0, str(Path.cwd().parent))

from backend.ai_judge_framework import GenericCriteriaJudge

# Load environment variables
load_dotenv()

# Verify API key
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not found in environment variables"
print("✅ Setup complete. API key verified.")

## Bias Type 1: Self-Preference Bias

**Definition**: Judges favor outputs generated by their own model.

**Detection Method**: Compare same-model judgments (GPT-4o judging GPT-4o outputs) vs cross-model judgments (GPT-4o judging Claude outputs).

**Expected Behavior**: Judges should be equally critical of outputs regardless of source model.

In [ ]:
# Test data for self-preference bias
# We'll use identical-quality responses but label them as from different models

test_queries_self_bias = [
    {
        "query": "How do I make chocolate chip cookies?",
        "response": "Mix butter, sugar, eggs, and flour. Add chocolate chips. Bake at 350F for 12 minutes.",
        "model_a": "gpt-4o",
        "model_b": "claude-sonnet"
    },
    {
        "query": "What's the best way to cook salmon?",
        "response": "Season with salt and pepper. Bake at 400F for 12-15 minutes until flaky.",
        "model_a": "gpt-4o",
        "model_b": "claude-sonnet"
    },
    {
        "query": "How long to boil pasta?",
        "response": "Boil for 8-10 minutes for al dente, or follow package instructions.",
        "model_a": "gpt-4o",
        "model_b": "claude-sonnet"
    },
    {
        "query": "What temperature for roasting vegetables?",
        "response": "Roast at 425F for 25-30 minutes, tossing halfway through.",
        "model_a": "gpt-4o",
        "model_b": "claude-sonnet"
    },
    {
        "query": "How to make scrambled eggs?",
        "response": "Beat eggs with milk. Cook in butter over medium heat, stirring gently.",
        "model_a": "gpt-4o",
        "model_b": "claude-sonnet"
    },
    {
        "query": "What's the secret to fluffy pancakes?",
        "response": "Don't overmix the batter and let it rest for 5 minutes before cooking.",
        "model_a": "gpt-4o",
        "model_b": "claude-sonnet"
    },
    {
        "query": "How to properly season a steak?",
        "response": "Apply salt and pepper generously to both sides at least 40 minutes before cooking.",
        "model_a": "gpt-4o",
        "model_b": "claude-sonnet"
    },
    {
        "query": "What's the ideal coffee brewing temperature?",
        "response": "Water should be between 195-205F for optimal extraction.",
        "model_a": "gpt-4o",
        "model_b": "claude-sonnet"
    },
    {
        "query": "How long to rest meat after cooking?",
        "response": "Let meat rest for 5-10 minutes to allow juices to redistribute.",
        "model_a": "gpt-4o",
        "model_b": "claude-sonnet"
    },
    {
        "query": "What's the best way to reheat pizza?",
        "response": "Reheat in a skillet over medium heat for 2-3 minutes for crispy crust.",
        "model_a": "gpt-4o",
        "model_b": "claude-sonnet"
    }
]

test_queries_self_bias = test_queries_self_bias[:NUM_COMPARISONS]

# Create judge for helpfulness (binary criterion)
judge_gpt4o = GenericCriteriaJudge(
    model="gpt-4o-mini",
    criteria="helpfulness",
    criteria_description="The response MUST fully address the user's query with sufficient detail.",
    temperature=0.0
)

# Test self-preference: Same response, but tell judge it's from different models
print(f"Testing self-preference bias with {len(test_queries_self_bias)} queries...\n")

self_bias_results = []

for i, example in enumerate(test_queries_self_bias, 1):
    # Judge when told it's from GPT-4o (same model)
    query_gpt4o = f"{example['query']} [Generated by {example['model_a']}]"
    result_gpt4o = judge_gpt4o.evaluate(query=query_gpt4o, response=example['response'])
    score_gpt4o = 1 if result_gpt4o.score == "PASS" else 0
    
    # Judge when told it's from Claude (different model)
    query_claude = f"{example['query']} [Generated by {example['model_b']}]"
    result_claude = judge_gpt4o.evaluate(query=query_claude, response=example['response'])
    score_claude = 1 if result_claude.score == "PASS" else 0
    
    self_bias_results.append({
        "query": example['query'],
        "score_same_model": score_gpt4o,
        "score_different_model": score_claude,
        "bias_detected": score_gpt4o > score_claude
    })
    
    print(f"{i}. Same model: {score_gpt4o}, Different model: {score_claude}")

# Calculate bias
avg_score_same = np.mean([r['score_same_model'] for r in self_bias_results])
avg_score_diff = np.mean([r['score_different_model'] for r in self_bias_results])
bias_magnitude = avg_score_same - avg_score_diff

print(f"\n{'='*60}")
print(f"SELF-PREFERENCE BIAS RESULTS")
print(f"{'='*60}")
print(f"Average score (same model):      {avg_score_same:.3f}")
print(f"Average score (different model): {avg_score_diff:.3f}")
print(f"Bias magnitude:                  {bias_magnitude:.3f}")
print(f"{'='*60}")

if abs(bias_magnitude) > 0.1:
    print(f"⚠️  BIAS DETECTED: Judge shows preference of {abs(bias_magnitude):.1%}")
else:
    print(f"✅ NO SIGNIFICANT BIAS: Judge is relatively unbiased")

## Bias Type 2: Position Bias

**Definition**: Judges favor the first or second position in A/B comparisons.

**Detection Method**: Present same pair in both orders (A vs B, then B vs A) and measure consistency.

**Expected Behavior**: Judge should give same verdict regardless of position order.

In [ ]:
# Test data for position bias
comparison_pairs = [
    {
        "query": "How do I make perfect rice?",
        "response_a": "Use 2:1 water to rice ratio. Bring to boil, then simmer covered for 18 minutes.",
        "response_b": "Rinse rice, use 1.5:1 water ratio, cook for 15 minutes, let rest 5 minutes."
    },
    {
        "query": "What's the best way to poach eggs?",
        "response_a": "Bring water to gentle simmer, add vinegar, create whirlpool, add egg, cook 3-4 minutes.",
        "response_b": "Boil water with vinegar, crack egg into cup, slide into water, cook 3 minutes."
    },
    {
        "query": "How to make crispy bacon?",
        "response_a": "Bake at 400F for 15-20 minutes on a wire rack over a baking sheet.",
        "response_b": "Cook in skillet over medium heat, flipping every 2 minutes until crispy."
    },
    {
        "query": "What's the secret to moist cake?",
        "response_a": "Don't overmix batter, use room temperature ingredients, don't overbake.",
        "response_b": "Add sour cream or yogurt to batter, and brush with simple syrup after baking."
    },
    {
        "query": "How to properly caramelize onions?",
        "response_a": "Cook over low heat for 30-40 minutes, stirring occasionally.",
        "response_b": "Slice thinly, cook over medium-low heat for 45 minutes with butter and salt."
    },
    {
        "query": "What's the best method for hard-boiled eggs?",
        "response_a": "Boil for 10 minutes, then immediately transfer to ice bath.",
        "response_b": "Steam for 12 minutes, cool in ice water for easy peeling."
    },
    {
        "query": "How to make fluffy mashed potatoes?",
        "response_a": "Use Yukon Gold potatoes, mash with butter and warm milk.",
        "response_b": "Boil russet potatoes, rice them, fold in cream and melted butter."
    },
    {
        "query": "What's the ideal way to grill chicken?",
        "response_a": "Marinate for 2 hours, grill over medium-high heat for 6-7 minutes per side.",
        "response_b": "Brine for 30 minutes, grill at 400F for 12-15 minutes total."
    },
    {
        "query": "How to make perfect guacamole?",
        "response_a": "Mash avocados with lime juice, add diced tomatoes, onion, cilantro, and salt.",
        "response_b": "Roughly mash avocados, mix in lime, jalapeño, cilantro, and salt to taste."
    },
    {
        "query": "What's the best way to store fresh herbs?",
        "response_a": "Trim stems, place in water like flowers, cover loosely with plastic bag.",
        "response_b": "Wrap in damp paper towel, store in sealed container in refrigerator."
    }
]

comparison_pairs = comparison_pairs[:NUM_COMPARISONS]

# Create pairwise comparison judge
def compare_responses(query: str, response_a: str, response_b: str) -> str:
    """Compare two responses and return which is better (A or B)."""
    prompt = f"""
You are evaluating recipe responses for helpfulness.

Query: {query}

Response A: {response_a}

Response B: {response_b}

Which response is MORE helpful and complete? Respond with JSON:
{{
    "winner": "A" or "B",
    "reasoning": "Brief explanation"
}}
"""
    
    response = litellm.completion(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    
    content = response.choices[0].message.content
    
    # Parse JSON
    if "```json" in content:
        json_start = content.find("```json") + 7
        json_end = content.find("```", json_start)
        json_text = content[json_start:json_end].strip()
    elif "{" in content:
        json_start = content.find("{")
        json_end = content.rfind("}") + 1
        json_text = content[json_start:json_end]
    else:
        json_text = content
    
    result = json.loads(json_text)
    return result["winner"]

# Test position bias
print(f"Testing position bias with {len(comparison_pairs)} pairs...\n")

position_bias_results = []

for i, pair in enumerate(comparison_pairs, 1):
    # Original order: A vs B
    winner_ab = compare_responses(pair['query'], pair['response_a'], pair['response_b'])
    
    # Swapped order: B vs A
    winner_ba = compare_responses(pair['query'], pair['response_b'], pair['response_a'])
    
    # Convert to consistent scoring (1 = A wins, 0 = B wins)
    score_ab = 1 if winner_ab == "A" else 0
    score_ba = 1 if winner_ba == "B" else 0  # Note: B is now in position A
    
    consistent = (score_ab == score_ba)
    
    position_bias_results.append({
        "query": pair['query'],
        "winner_ab": winner_ab,
        "winner_ba": winner_ba,
        "consistent": consistent
    })
    
    print(f"{i}. A vs B: {winner_ab} wins | B vs A: {winner_ba} wins | Consistent: {consistent}")

# Calculate bias
consistency_rate = np.mean([r['consistent'] for r in position_bias_results])
first_position_wins = np.mean([r['winner_ab'] == 'A' for r in position_bias_results])

print(f"\n{'='*60}")
print(f"POSITION BIAS RESULTS")
print(f"{'='*60}")
print(f"Consistency rate:          {consistency_rate:.1%}")
print(f"First position win rate:   {first_position_wins:.1%}")
print(f"{'='*60}")

if consistency_rate < 0.8:
    print(f"⚠️  POSITION BIAS DETECTED: Only {consistency_rate:.0%} consistency")
else:
    print(f"✅ NO SIGNIFICANT BIAS: High consistency ({consistency_rate:.0%})")

if abs(first_position_wins - 0.5) > 0.2:
    print(f"⚠️  FIRST-POSITION BIAS: {first_position_wins:.0%} of first-position responses win")
else:
    print(f"✅ BALANCED: First position wins ~50% of the time")

## Bias Type 3: Verbosity Bias

**Definition**: Judges favor longer responses regardless of quality.

**Detection Method**: Correlate response length with judge scores.

**Expected Behavior**: Response length should not correlate with quality scores when content quality is controlled.

In [ ]:
# Test data for verbosity bias (varying lengths with similar quality)
verbosity_test_data = [
    {
        "query": "How do I make scrambled eggs?",
        "response": "Beat eggs. Cook in butter. Stir until set.",
        "length": 41
    },
    {
        "query": "How do I make scrambled eggs?",
        "response": "Beat eggs with a splash of milk. Pour into a hot buttered pan. Stir gently over medium heat until just set.",
        "length": 115
    },
    {
        "query": "How do I make scrambled eggs?",
        "response": "To make perfect scrambled eggs, start by cracking your eggs into a bowl and whisking them thoroughly with a small splash of milk or cream. Heat a non-stick pan over medium heat and add a tablespoon of butter. Once the butter is melted and foaming, pour in your egg mixture. Using a silicone spatula, gently push the eggs from the edges toward the center, allowing the uncooked eggs to flow to the edges. Continue this process, stirring gently and slowly, until the eggs are just set but still slightly creamy. Remove from heat immediately and season with salt and pepper to taste.",
        "length": 569
    },
    {
        "query": "What temperature to bake cookies?",
        "response": "350F for 12 minutes.",
        "length": 22
    },
    {
        "query": "What temperature to bake cookies?",
        "response": "Bake at 350 degrees Fahrenheit for 10-12 minutes until edges are golden brown.",
        "length": 81
    },
    {
        "query": "What temperature to bake cookies?",
        "response": "For optimal chocolate chip cookie results, preheat your oven to 350 degrees Fahrenheit (175 degrees Celsius). This moderate temperature allows the cookies to bake evenly, with the edges becoming golden and slightly crispy while the centers remain soft and chewy. Baking time typically ranges from 10 to 12 minutes, depending on your oven and cookie size. Watch for visual cues: the edges should be set and lightly golden, while the centers may still look slightly underdone. Remember, cookies continue cooking on the hot baking sheet even after removal from the oven, so don't overbake!",
        "length": 549
    },
    {
        "query": "How long to boil pasta?",
        "response": "8-10 minutes for al dente.",
        "length": 26
    },
    {
        "query": "How long to boil pasta?",
        "response": "Boil pasta for 8-10 minutes for al dente texture, or follow the package directions.",
        "length": 84
    },
    {
        "query": "How long to boil pasta?",
        "response": "The cooking time for pasta depends on the type and shape, but as a general guideline, most dried pasta should be boiled for 8-10 minutes to achieve the ideal al dente texture. Al dente means the pasta is cooked through but still has a slight firmness when bitten. To cook pasta properly, bring a large pot of salted water to a rolling boil, add the pasta, and stir occasionally to prevent sticking. Start testing the pasta a minute or two before the package's suggested cooking time by removing a piece and biting into it. The pasta should be tender but still offer a bit of resistance. Remember that pasta continues to cook slightly after draining, so it's better to err on the side of slightly underdone if you're adding it to a sauce.",
        "length": 679
    },
    {
        "query": "How to store fresh basil?",
        "response": "Trim stems, place in water, refrigerate.",
        "length": 42
    }
]

verbosity_test_data = verbosity_test_data[:NUM_COMPARISONS]

# Evaluate all responses
print(f"Testing verbosity bias with {len(verbosity_test_data)} responses...\n")

verbosity_bias_results = []

for i, example in enumerate(verbosity_test_data, 1):
    result = judge_gpt4o.evaluate(query=example['query'], response=example['response'])
    score = 1 if result.score == "PASS" else 0
    
    verbosity_bias_results.append({
        "query": example['query'],
        "length": example['length'],
        "score": score
    })
    
    print(f"{i}. Length: {example['length']:3d} chars | Score: {score}")

# Calculate correlation
lengths = [r['length'] for r in verbosity_bias_results]
scores = [r['score'] for r in verbosity_bias_results]

if len(set(scores)) > 1:  # Only calculate if there's variance in scores
    correlation, p_value = stats.pearsonr(lengths, scores)
else:
    correlation = 0.0
    p_value = 1.0

print(f"\n{'='*60}")
print(f"VERBOSITY BIAS RESULTS")
print(f"{'='*60}")
print(f"Correlation (length vs score): {correlation:.3f}")
print(f"P-value:                       {p_value:.3f}")
print(f"Average length:                {np.mean(lengths):.0f} chars")
print(f"Average score:                 {np.mean(scores):.3f}")
print(f"{'='*60}")

if correlation > 0.4 and p_value < 0.05:
    print(f"⚠️  VERBOSITY BIAS DETECTED: Strong positive correlation ({correlation:.2f})")
elif correlation > 0.3:
    print(f"⚠️  MODERATE VERBOSITY BIAS: Correlation = {correlation:.2f}")
else:
    print(f"✅ NO SIGNIFICANT BIAS: Low correlation ({correlation:.2f})")

## Visualization: Bias Patterns

Let's create visualizations for all three bias types.

In [ ]:
# Create 3-panel visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Panel 1: Self-Preference Bias
ax1 = axes[0]
categories = ['Same Model', 'Different Model']
scores = [avg_score_same, avg_score_diff]
colors = ['#ff6b6b' if avg_score_same > avg_score_diff else '#51cf66', '#4dabf7']
ax1.bar(categories, scores, color=colors, alpha=0.7, edgecolor='black')
ax1.set_ylabel('Average Score', fontsize=12)
ax1.set_title(f'Self-Preference Bias\nBias: {bias_magnitude:+.3f}', fontsize=14, fontweight='bold')
ax1.set_ylim([0, 1.1])
ax1.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
for i, v in enumerate(scores):
    ax1.text(i, v + 0.05, f'{v:.2f}', ha='center', fontweight='bold')

# Panel 2: Position Bias
ax2 = axes[1]
categories = ['Consistent', 'Inconsistent']
counts = [
    sum(r['consistent'] for r in position_bias_results),
    sum(not r['consistent'] for r in position_bias_results)
]
colors = ['#51cf66' if consistency_rate >= 0.8 else '#ff6b6b', '#ffa94d']
ax2.bar(categories, counts, color=colors, alpha=0.7, edgecolor='black')
ax2.set_ylabel('Number of Comparisons', fontsize=12)
ax2.set_title(f'Position Bias\nConsistency: {consistency_rate:.1%}', fontsize=14, fontweight='bold')
for i, v in enumerate(counts):
    ax2.text(i, v + 0.2, str(v), ha='center', fontweight='bold')

# Panel 3: Verbosity Bias
ax3 = axes[2]
ax3.scatter(lengths, scores, alpha=0.6, s=100, c=scores, cmap='RdYlGn', edgecolors='black')
ax3.set_xlabel('Response Length (characters)', fontsize=12)
ax3.set_ylabel('Judge Score', fontsize=12)
ax3.set_title(f'Verbosity Bias\nCorrelation: {correlation:.3f}', fontsize=14, fontweight='bold')
ax3.set_ylim([-0.1, 1.1])

# Add trend line if significant correlation
if abs(correlation) > 0.3:
    z = np.polyfit(lengths, scores, 1)
    p = np.poly1d(z)
    ax3.plot(lengths, p(lengths), "r--", alpha=0.8, linewidth=2, label=f'Trend (r={correlation:.2f})')
    ax3.legend()

plt.tight_layout()
plt.savefig('lesson-10/diagrams/bias_detection_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Bias visualization saved to lesson-10/diagrams/bias_detection_results.png")

## Mitigation Strategies

### How to Address Detected Biases

#### Self-Preference Bias
- ✅ **Use different model for judging**: Don't use GPT-4o to judge GPT-4o outputs
- ✅ **Blind evaluation**: Don't mention which model generated the response
- ✅ **Cross-validation**: Use multiple judges from different model families

#### Position Bias
- ✅ **Position swapping**: Always evaluate both orders (A vs B, B vs A) and aggregate
- ✅ **Randomize presentation**: Shuffle positions randomly for each comparison
- ✅ **Use absolute judgments**: Judge responses independently, not comparatively

#### Verbosity Bias
- ✅ **Length normalization**: Explicitly instruct judge to ignore length
- ✅ **Truncation testing**: Test with truncated versions to see if scores change
- ✅ **Prompt modification**: Add instruction like "Favor concise answers over verbose ones"

### Production Checklist

Before deploying judges in production:

- [ ] Test for self-preference bias with 20+ examples
- [ ] Test for position bias with 20+ pairs (evaluate both orders)
- [ ] Test for verbosity bias with length-controlled examples
- [ ] Measure consistency: Run same input multiple times
- [ ] Validate against ground truth: 50+ labeled examples
- [ ] Document bias detection results in evaluation report
- [ ] Implement mitigation strategies for detected biases
- [ ] Monitor bias metrics over time (track in production)

### Key Takeaways

1. **Biases are common**: Most judges exhibit some form of bias
2. **Detection is critical**: Don't assume your judge is unbiased
3. **Mitigation is possible**: Simple techniques (position swapping, blind evaluation) are effective
4. **Monitor continuously**: Biases can change as models are updated
5. **Document everything**: Keep records of bias tests for reproducibility

### Next Steps

- **Production Deployment**: Apply mitigation strategies to your judges
- **Continuous Monitoring**: Set up automated bias detection in production
- **Ensemble Judges**: Use multiple judges and aggregate their decisions
- **Human Validation**: Periodically validate judge decisions with human reviewers

👉 Continue to: [Lesson 11 - Comparative Evaluation & Leaderboards](../lesson-11/TUTORIAL_INDEX.md)